Text - Data

TODO: (briefly) describe dataset and cleaning

TODO: Please submit your cleaned data or, if it’s too large, a reference to the
original data as well as the scripts you used to clean it

In [ ]:
## Run code to clean data

Text - EDA #1 - Distribution of Reviews

TODO: Describe viz and takeaways

In [ ]:
## Run code to output viz

Text - EDA #2 - Heatmap of average score prices

TODO: Describe viz and takeaways

In [ ]:
## output viz #2

# TODO: Harket - create .py file to output viz

Text - EDA #3 - Average score vs price

TODO: Describe viz and takeaways

In [ ]:
## Output viz 3

## TODO: Sonya - Create .py file to output viz

Text - ML #1 - Binary classification of reviews

TODO: Describe study and takeaways

In [ ]:
## TODO: Load model, benchmark on test data, and evaluate on test data

Text - ML #2 - Multi-classification of reviews

TODO: Describe study and takeaways

In [ ]:
## TODO: Load model, benchmark on test data, and evaluate on test data

Text - App Demo

https://yelp-recs.crowsnet.io

TODO: Describe app and data pulling/cleaning process

Text - Results

1. Yelp data is biased
2. Yelp bias is user input, at least in part